In [1]:
# 목표 예측 범위 안에 넣기
# data * std + mean


In [2]:
%matplotlib widget

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
folder = 'data'
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
             ]

#file_names = [['자동측정망-01-가평-2016.xlsx','자동측정망-01-가평-2017.xlsx','자동측정망-01-가평-2018.xlsx', '자동측정망-01-가평-2019.xlsx', '자동측정망-01-가평-2020.xlsx'], 
#              ['자동측정망-02-의암호-2016.xlsx','자동측정망-02-의암호-2017.xlsx','자동측정망-02-의암호-2018.xlsx','자동측정망-02-의암호-2019.xlsx','자동측정망-02-의암호-2020.xlsx'],
#              ['자동측정망-03-서상-2016.xlsx','자동측정망-03-서상-2017.xlsx','자동측정망-03-서상-2018.xlsx','자동측정망-03-서상-2019.xlsx','자동측정망-03-서상-2020.xlsx'],
#              ['자동측정망-04-화천-2016.xlsx','자동측정망-04-화천-2017.xlsx','자동측정망-04-화천-2018.xlsx','자동측정망-04-화천-2019.xlsx','자동측정망-04-화천-2020.xlsx'],
              #['총량측정망-10-01-조종천3-2016.xlsx','총량측정망-10-01-조종천3-2017.xlsx','총량측정망-10-01-조종천3-2018.xlsx','총량측정망-10-01-조종천3-2019.xlsx','총량측정망-10-01-조종천3-2020.xlsx'],
              #['총량측정망-12-01-청평-2016.xlsx','총량측정망-12-01-청평-2017.xlsx','총량측정망-12-01-청평-2018.xlsx','총량측정망-12-01-청평-2019.xlsx','총량측정망-12-01-청평-2020.xlsx'],
              #['총량측정망-12-01-청평-2016s.xlsx','총량측정망-12-01-청평-2017.xlsx','총량측정망-12-01-청평-2018.xlsx','총량측정망-12-01-청평-2019.xlsx','총량측정망-12-01-청평-2020.xlsx'],
              #['총량측정망-12-02-가평천3-2016.xlsx','총량측정망-12-02-가평천3-2017.xlsx','총량측정망-12-02-가평천3-2018.xlsx','총량측정망-12-02-가평천3-2019.xlsx','총량측정망-12-02-가평천3-2020.xlsx'],
              #['총량측정망-12-03-춘성교-2016.xlsx','총량측정망-12-03-춘성교-2017.xlsx','총량측정망-12-03-춘성교-2018.xlsx','총량측정망-12-03-춘성교-2019.xlsx','총량측정망-12-03-춘성교-2020.xlsx'],
              #['총량측정망-20-01-소양강2-2016.xlsx','총량측정망-20-01-소양강2-2017.xlsx','총량측정망-20-01-소양강2-2018.xlsx','총량측정망-20-01-소양강2-2019.xlsx','총량측정망-20-01-소양강2-2020.xlsx'],
              #['총량측정망-34-01-춘천A-2016.xlsx','총량측정망-34-01-춘천A-2017.xlsx','총량측정망-34-01-춘천A-2018.xlsx','총량측정망-34-01-춘천A-2019.xlsx','총량측정망-34-01-춘천A-2020.xlsx'],
              #['총량측정망-40-01-화천-2016.xlsx','총량측정망-40-01-화천-2017.xlsx','총량측정망-40-01-화천-2018.xlsx','총량측정망-40-01-화천-2019.xlsx','총량측정망-40-01-화천-2020.xlsx']
         #    ]



day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

stride = len(file_names)

for loc in range(len(file_names)):
    
    df_loc = []
    print(len(file_names[loc]))
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].iloc[:, 2:])
#     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
#     df.append(df_full[loc].iloc[:, 2:11])
    
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
        

4
가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
4
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx


In [5]:
#df[0]

In [6]:
#df[0]

In [7]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [8]:
train_mean, train_std


(수온          1.442927e+01
 수소이온농도      7.338229e+00
 전기전도도       1.067481e+02
 용존산소        1.051423e+01
 탁도          2.159194e+00
 총유기탄소       1.680518e+00
 총질소         1.717054e+00
 총인          7.725232e-03
 클로로필-a      1.064802e+01
 미생물_독성지수   -4.499445e-02
 Day sin     6.037379e-15
 Day cos     2.747517e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           7.397900
 수소이온농도       0.485797
 전기전도도       13.159411
 용존산소         1.806036
 탁도           5.437522
 총유기탄소        0.368691
 총질소          0.382042
 총인           0.007932
 클로로필-a      10.570897
 미생물_독성지수     1.615261
 Day sin      0.707112
 Day cos      0.707112
 Year sin     0.707105
 Year cos     0.707119
 dtype: float64)

In [9]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [10]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [11]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [12]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [13]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [14]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [15]:
# not used
# original make_dataset code
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

#WindowGenerator.make_dataset = make_dataset

In [16]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [17]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [18]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [19]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [20]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [21]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        
        #print(data)
        
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [22]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [23]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [24]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

dgen = GainDataGenerator(df)

In [25]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [26]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [27]:
train_df = df_all
val_df = df_all
test_df = df_all

In [28]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [29]:
wide_window.example[0].shape

TensorShape([128, 120, 14])

In [30]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [31]:
wide_window.plot(plot_col='총유기탄소')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  2173


In [32]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = ~isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:7], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure(figsize=(9,10))
n = wide_window.dg.data_m.shape[0]
train = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(wide_window.dg.data_m[i*train:(i+1)*train, 0:7], aspect='auto')
    plt.yticks([])
#plt.imshow(wide_window.dg.data[0:100])
#plt.imshow(wide_window.dg.data_m[800:900], aspect='auto')
#print(wide_window.dg.data[0:50])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [34]:
val_performance = {}
performance = {}

In [35]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [36]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [37]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

gain.save(save_dir='save')

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 100.7219 - disc_loss: 0.7219 - rmse: 0.9108 - val_loss: 0.9624
Epoch 2/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 96.5649 - disc_loss: 0.5047 - rmse: 0.9140 - val_loss: 0.7899
Epoch 3/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 72.3257 - disc_loss: 0.5301 - rmse: 0.7819 - val_loss: 0.7391
Epoch 4/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 63.7034 - disc_loss: 0.4084 - rmse: 0.7435 - val_loss: 0.6747
Epoch 5/2000
1/1 [==============================] - 0s 181ms/step - gen_loss: 57.2585 - disc_loss: 0.4208 - rmse: 0.6653 - val_loss: 0.6522
Epoch 6/2000
1/1 [==============================] - 0s 182ms/step - gen_loss: 54.9016 - disc_loss: 0.4354 - rmse: 0.6603 - val_loss: 0.5972
Epoch 7/2000
1/1 [==============================] - 0s 178ms/step - gen_loss: 41.4087 - disc_loss: 0.3955 - rmse: 0.5793 - val_loss: 0.5857
Epoch 8/2000
1/1 [====

1/1 [==============================] - 0s 174ms/step - gen_loss: 7.0261 - disc_loss: 0.2387 - rmse: 0.6189 - val_loss: 0.3210
Epoch 118/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 6.7115 - disc_loss: 0.2273 - rmse: 0.3411 - val_loss: 0.2887
Epoch 119/2000
1/1 [==============================] - 0s 365ms/step - gen_loss: 6.3600 - disc_loss: 0.2352 - rmse: 0.5231 - val_loss: 0.2865
Epoch 120/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 6.9186 - disc_loss: 0.2716 - rmse: 0.4120 - val_loss: 0.2838
Epoch 121/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 6.6404 - disc_loss: 0.2684 - rmse: 0.4026 - val_loss: 0.2954
Epoch 122/2000
1/1 [==============================] - 0s 176ms/step - gen_loss: 7.2026 - disc_loss: 0.2654 - rmse: 0.2985 - val_loss: 0.3305
Epoch 123/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 7.0179 - disc_loss: 0.2706 - rmse: 0.2971 - val_loss: 0.3109
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 170ms/step - gen_loss: 5.4571 - disc_loss: 0.1979 - rmse: 0.3860 - val_loss: 0.4071
Epoch 234/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 5.9183 - disc_loss: 0.2019 - rmse: 0.4079 - val_loss: 0.3767
Epoch 235/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 5.3956 - disc_loss: 0.1945 - rmse: 0.3928 - val_loss: 0.5602
Epoch 236/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 5.2591 - disc_loss: 0.1987 - rmse: 0.2734 - val_loss: 0.2981
Epoch 237/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 4.9774 - disc_loss: 0.1984 - rmse: 0.4466 - val_loss: 0.3154
Epoch 238/2000
1/1 [==============================] - 0s 382ms/step - gen_loss: 6.3746 - disc_loss: 0.1955 - rmse: 0.3969 - val_loss: 0.4704
Epoch 239/2000
1/1 [==============================] - 0s 182ms/step - gen_loss: 6.8922 - disc_loss: 0.2007 - rmse: 0.2738 - val_loss: 0.3163
Epoch 240/2000
1/1 [========

1/1 [==============================] - 0s 190ms/step - gen_loss: 6.0455 - disc_loss: 0.1797 - rmse: 0.2916 - val_loss: 0.2881
Epoch 350/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 4.7137 - disc_loss: 0.1909 - rmse: 0.3049 - val_loss: 0.2816
Epoch 351/2000
1/1 [==============================] - 0s 176ms/step - gen_loss: 6.2144 - disc_loss: 0.1826 - rmse: 0.2995 - val_loss: 0.2819
Epoch 352/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 5.2930 - disc_loss: 0.1798 - rmse: 0.2826 - val_loss: 0.2670
Epoch 353/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 7.1535 - disc_loss: 0.1850 - rmse: 0.2826 - val_loss: 0.2804
Epoch 354/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 4.7069 - disc_loss: 0.1732 - rmse: 0.2894 - val_loss: 0.3031
Epoch 355/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 6.4981 - disc_loss: 0.1826 - rmse: 0.2944 - val_loss: 0.3030
Epoch 356/2000
1/1 [========

1/1 [==============================] - 0s 171ms/step - gen_loss: 4.2459 - disc_loss: 0.1651 - rmse: 0.3392 - val_loss: 0.4731
Epoch 466/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 4.1324 - disc_loss: 0.1660 - rmse: 0.3428 - val_loss: 0.3763
Epoch 467/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 4.8514 - disc_loss: 0.1635 - rmse: 0.5017 - val_loss: 0.2897
Epoch 468/2000
1/1 [==============================] - 0s 184ms/step - gen_loss: 4.4031 - disc_loss: 0.1701 - rmse: 0.4253 - val_loss: 0.4608
Epoch 469/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 4.5107 - disc_loss: 0.1663 - rmse: 0.3803 - val_loss: 0.4584
Epoch 470/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 4.5252 - disc_loss: 0.1597 - rmse: 0.3318 - val_loss: 0.5917
Epoch 471/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 4.4407 - disc_loss: 0.1681 - rmse: 0.3852 - val_loss: 0.5321
Epoch 472/2000
1/1 [========

...

1/1 [==============================] - 0s 168ms/step - gen_loss: 2.0366 - disc_loss: 0.1288 - rmse: 0.2567 - val_loss: 0.2723
Epoch 1502/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 1.8183 - disc_loss: 0.1280 - rmse: 0.2366 - val_loss: 0.2918
Epoch 1503/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 1.9936 - disc_loss: 0.1239 - rmse: 0.2829 - val_loss: 0.2898
Epoch 1504/2000
1/1 [==============================] - 0s 164ms/step - gen_loss: 2.0810 - disc_loss: 0.1267 - rmse: 0.2376 - val_loss: 0.2645
Epoch 1505/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 1.9340 - disc_loss: 0.1255 - rmse: 0.2222 - val_loss: 0.2879
Epoch 1506/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 1.9024 - disc_loss: 0.1263 - rmse: 0.2495 - val_loss: 0.2978
Epoch 1507/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 1.9629 - disc_loss: 0.1214 - rmse: 0.2878 - val_loss: 0.2818
Epoch 1508/2000
1/1 [=

1/1 [==============================] - 0s 166ms/step - gen_loss: 1.8925 - disc_loss: 0.1237 - rmse: 0.2456 - val_loss: 0.2434
Epoch 1616/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 1.9010 - disc_loss: 0.1313 - rmse: 0.2627 - val_loss: 0.2513
Epoch 1617/2000
1/1 [==============================] - 0s 164ms/step - gen_loss: 1.8204 - disc_loss: 0.1286 - rmse: 0.2607 - val_loss: 0.2538
Epoch 1618/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 2.0259 - disc_loss: 0.1276 - rmse: 0.2717 - val_loss: 0.2522
Epoch 1619/2000
1/1 [==============================] - 0s 181ms/step - gen_loss: 2.0625 - disc_loss: 0.1257 - rmse: 0.2952 - val_loss: 0.2434
Epoch 1620/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.8180 - disc_loss: 0.1286 - rmse: 0.2308 - val_loss: 0.2337
Epoch 1621/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 2.3953 - disc_loss: 0.1204 - rmse: 0.2618 - val_loss: 0.2639
Epoch 1622/2000
1/1 [=

1/1 [==============================] - 0s 167ms/step - gen_loss: 2.0448 - disc_loss: 0.1151 - rmse: 0.2567 - val_loss: 0.2453
Epoch 1730/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 1.8922 - disc_loss: 0.1169 - rmse: 0.2631 - val_loss: 0.2653
Epoch 1731/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.9107 - disc_loss: 0.1156 - rmse: 0.2586 - val_loss: 0.2556
Epoch 1732/2000
1/1 [==============================] - 0s 362ms/step - gen_loss: 1.8746 - disc_loss: 0.1260 - rmse: 0.2526 - val_loss: 0.2543
Epoch 1733/2000
1/1 [==============================] - 0s 180ms/step - gen_loss: 1.9563 - disc_loss: 0.1278 - rmse: 0.2576 - val_loss: 0.2499
Epoch 1734/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 2.2134 - disc_loss: 0.1241 - rmse: 0.3891 - val_loss: 0.2710
Epoch 1735/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 1.9566 - disc_loss: 0.1251 - rmse: 0.2969 - val_loss: 0.2623
Epoch 1736/2000
1/1 [=

1/1 [==============================] - 0s 170ms/step - gen_loss: 1.8001 - disc_loss: 0.1184 - rmse: 0.3529 - val_loss: 0.2453
Epoch 1844/2000
1/1 [==============================] - 0s 180ms/step - gen_loss: 1.7159 - disc_loss: 0.1198 - rmse: 0.3546 - val_loss: 0.3475
Epoch 1845/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 1.5453 - disc_loss: 0.1193 - rmse: 0.3723 - val_loss: 0.4210
Epoch 1846/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 1.7497 - disc_loss: 0.1195 - rmse: 0.2348 - val_loss: 0.4724
Epoch 1847/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 1.6239 - disc_loss: 0.1165 - rmse: 0.2256 - val_loss: 0.3352
Epoch 1848/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 1.7956 - disc_loss: 0.1190 - rmse: 0.5171 - val_loss: 0.2358
Epoch 1849/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 1.7027 - disc_loss: 0.1194 - rmse: 0.3731 - val_loss: 0.2891
Epoch 1850/2000
1/1 [=

1/1 [==============================] - 0s 167ms/step - gen_loss: 1.8030 - disc_loss: 0.1206 - rmse: 0.2412 - val_loss: 0.2453
Epoch 1958/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 1.7570 - disc_loss: 0.1170 - rmse: 0.2289 - val_loss: 0.2720
Epoch 1959/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 1.5446 - disc_loss: 0.1196 - rmse: 0.2203 - val_loss: 0.2497
Epoch 1960/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 1.7191 - disc_loss: 0.1186 - rmse: 0.2203 - val_loss: 0.2815
Epoch 1961/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 1.8491 - disc_loss: 0.1165 - rmse: 0.2286 - val_loss: 0.2651
Epoch 1962/2000
1/1 [==============================] - 0s 179ms/step - gen_loss: 1.7463 - disc_loss: 0.1166 - rmse: 0.2237 - val_loss: 0.2690
Epoch 1963/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.5231 - disc_loss: 0.1225 - rmse: 0.2161 - val_loss: 0.2676
Epoch 1964/2000
1/1 [=

**학습 loss history 출력**

In [38]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

성능 측정

In [39]:
gain.evaluate(wide_window.test.repeat(), steps=100)

100/100 [==============================] - 3s 31ms/step - loss: 0.3231


0.3231378495693207

샘플 prediction 출력

In [40]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
norm_df = pd.concat(df,axis=0)

In [42]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

70128
(120, 14)
120
x.shape = (70080, 14)
x_reshape.shape = (584, 120, 14)
136398
y_true.shape= (70080, 14)
136398


(1, 120, 14)

In [43]:
# zero loss is normal because there is no ground truth in the real dataset
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

19/19 [==============================] - 0s 6ms/step - loss: 0.0000e+00


0.0

In [44]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [45]:
print(total_n)
print(n)
print(total_n-n)

70128
70080
48


In [46]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(70080, 14) (120, 14)
(70128, 14)


In [47]:
y_pred[~np.isnan(data)] = np.nan

In [48]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

70128
unit_shape= (120, 14)
120
n= 70080
(35040, 10)
(35040, 10)


## 예측 

In [50]:
total_no = oris[0].shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gans[0][train_slice])
val_df = pd.DataFrame(gans[0][val_slice])
test_df = pd.DataFrame(gans[0][test_slice])

train_ori_df = pd.DataFrame(oris[0][train_slice])
val_ori_df = pd.DataFrame(oris[0][val_slice])
test_ori_df = pd.DataFrame(oris[0][test_slice])

num_features = train_df.shape[1]
out_num_features = num_features

In [51]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features


(35040, 14) (35040, 14)
(35040, 10) (35040, 10)
(35040, 24) (35040, 24)


1

In [52]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [53]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features
                        )

In [54]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,...,1.414204,-1.999581e-12,1.414155,-0.003771,-0.005978,-0.078693,-1.531417,0.324341,0.703062,-0.218389
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,...,1.366016,3.660228e-01,1.414157,-0.002757,0.037560,-0.078693,-1.531417,0.324341,0.703062,-0.218389
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,...,1.224736,7.071018e-01,1.414159,-0.001743,0.040280,-0.078693,-1.531417,0.268971,0.703062,-0.218389
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,...,0.999993,9.999928e-01,1.414160,-0.000730,-0.023549,-0.078693,-1.544935,0.268971,0.703062,-0.218389
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,...,0.707102,1.224736e+00,1.414160,0.000284,-0.003100,-0.078693,-1.544935,0.324341,0.703062,-0.218389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.293425,-0.902082,0.627070,-0.672317,0.007504,0.324071,0.913370,0.16071,-0.401860,-0.095963,...,0.366023,-1.366016e+00,0.412794,-1.352621,-0.088405,-0.696235,0.158251,-0.340098,-0.436807,0.595301
24524,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,0.994513,0.03464,-0.439699,-0.343601,...,0.707102,-1.224736e+00,0.413763,-1.352324,-0.153610,-0.696235,0.158251,-0.340098,-0.436807,0.324071
24525,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,1.004983,0.16071,-0.439699,-0.343601,...,0.999993,-9.999928e-01,0.414732,-1.352028,-0.123704,-0.696235,0.158251,-0.340098,-0.360816,0.324071
24526,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,1.012835,0.16071,-0.439699,-0.095963,...,1.224736,-7.071018e-01,0.415701,-1.351730,-0.094411,-0.696235,0.144734,-0.340098,-0.360816,0.324071


In [55]:
it = iter(wdg)

In [56]:
x,y = next(it)
x.shape, y.shape

((128, 168, 24), (128, 72, 1))

In [57]:
def make_dataset_water(self, data):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [58]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [59]:
def hour_to_day_mean(array):
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

In [60]:

cond = {
    "총인":[
        [-1000,0.02],
        [0.02,0.04],
        [0.04,0.10],
        [0.10,0.20],
        [0.20,0.30],
        [0.30,0.50],
        [0.50,1000],
    ],
    "총질소":[
        [-1000,0.2],
        [0.2,0.3],
        [0.3,0.4],
        [0.4,0.6],
        [0.6,1.0],
        [1.0,1.5],
        [1.5,1000],
    ],
    "수소이온농도":[
        [6.5 ,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.0,8.5],
        [6.0,8.5],
        [-1000,1000],
    ],
    "총유기탄소":[
        [-1000 ,2],
        [2,3],
        [3,4],
        [4,5],
        [5,6],
        [6,8],
        [8,1000],
    ],
    "용존산소":[
        [7.5 ,1000],
        [5,7.5],
        [5,7.5],
        [5,7.5],
        [2,5],
        [2,5],
        [-1000,2],
    ],
    "클로로필-a":[
        [-1000 ,5],
        [5,9],
        [9,14],
        [14,20],
        [20,42],
        [42,70],
        [70,1000],
    ],
    
}


def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  count1 = 0
  count2 = 0
  for n in range(len(inputs)):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
    pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    level_label = 0 
    level_pred = 0
    
    for i in range(0,7,1):
      min, max = cond[df_all.columns[plot_col]][i]
      if min <= label_temp[2] and label_temp[2] < max:
        level_label = i
            #print(i, label_temp[2])
      if min <= pred_temp[2] and pred_temp[2] < max:
        level_pred = i
            
    if level_pred != level_label:
        count1 = count1 + 1
    count2 = count2 + 1 
        
  print(100 - count1*100/count2)

  return 100 - count1*100/count2
    

WindowGenerator.compa = compa

In [61]:

def to_class(value, colum):
    level_label = 0
    for i in range(0,7,1):
        min1, max1 = cond[colum][i]
        if min1 <= value and value < max1:
            level_label = i
            # 반전
            #level_label = 7 - i
            return level_label
    return level_label

def plot24_class(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    plt.ylim((0,8))
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    input_temp = input_temp.numpy()
    for day in range(input_temp.size):
        #level_label = 0
        #for i in range(0,7,1):
         #   min1, max1 = cond[df_all.columns[plot_col]][i]
          #  if min1 <= input_temp[0] and input_temp[0] < max1:
          #      level_label = i
          #      break; 
        #input_temp[day] = level_label
        input_temp[day] = to_class(input_temp[day], df_all.columns[plot_col])
                
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    label_temp = label_temp.numpy()
    for day in range(label_temp.size):
        label_temp[day] = to_class(label_temp[day], df_all.columns[plot_col])
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
        
      predict_temp = predict_temp.numpy()
      for day in range(predict_temp.size):
          predict_temp[day] = to_class(predict_temp[day], df_all.columns[plot_col])
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24_class = plot24_class

#multi_window.plot24_class(last_baseline, plot_col=out_features[0])

In [62]:

def plot_class(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  #plt.figure(figsize=(10, 8))
  plt.figure()
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  
  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    plt.ylim((0,8))
    
    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    label_temp = label_temp.numpy()
    for day in range(label_temp.size):
        label_temp[day] = to_class(label_temp[day], df_all.columns[plot_col])
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
        
      predict_temp = predict_temp.numpy()
      for day in range(predict_temp.size):
          predict_temp[day] = to_class(predict_temp[day], df_all.columns[plot_col])

    bar_label = [label_temp[v] for v in range(label_temp.size)]
    bar_pred = [predict_temp[v] for v in range(predict_temp.size)]
        
    x = np.arange(3)
    width = 0.3
    plt.bar(x - 0.17, bar_label, width, label='label')
    plt.bar(x + 0.17, bar_pred, width, label='pred')
    plt.xticks(ticks=[0,1,2], labels=[1,2,3],rotation=0)
    if n == 0:
        plt.legend()

WindowGenerator.plot_class = plot_class

In [63]:
df_all.columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       '미생물_독성지수', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [64]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
#multi_window

In [65]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [66]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [67]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


100/100 [==============================] - 6s 64ms/step - loss: 0.0644 - mean_absolute_error: 0.1512
val performance = [0.06559404730796814, 0.1519269198179245]
test performance =  [0.21696370840072632, 0.23474936187267303]


In [68]:
out_features = [8]
out_features

[8]

In [69]:
_performance={}
#multi_window.plot24(last_baseline, plot_col=out_features[0])
#multi_window.plot24(last_baseline, plot_col='총유기탄소')

_performance['base'] = multi_window.compa(last_baseline, plot_col=out_features[0])

column : 클로로필-a
61.71875


In [70]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
multi_window.plot24(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
#multi_window.plot24_class(last_baseline, plot_col=out_features[0])
multi_window.plot_class(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Linear**

In [73]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/400
1/1 [==============================] - 0s 381ms/step - loss: 0.5483 - mean_absolute_error: 0.5303 - val_loss: 0.2938 - val_mean_absolute_error: 0.4690
Epoch 2/400
1/1 [==============================] - 0s 164ms/step - loss: 0.6578 - mean_absolute_error: 0.5848 - val_loss: 0.3438 - val_mean_absolute_error: 0.5086
Epoch 3/400
1/1 [==============================] - 0s 161ms/step - loss: 0.4591 - mean_absolute_error: 0.4945 - val_loss: 0.3265 - val_mean_absolute_error: 0.5103
Epoch 4/400
1/1 [==============================] - 0s 157ms/step - loss: 0.4941 - mean_absolute_error: 0.5043 - val_loss: 0.3262 - val_mean_absolute_error: 0.5062
Epoch 5/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3580 - mean_absolute_error: 0.4730 - val_loss: 0.2735 - val_mean_absolute_error: 0.4511
Epoch 6/400
1/1 [==============================] - 0s 158ms/step - loss: 0.5884 - mean_absolute_error: 0.4966 - val_loss: 0.2955 - val_mean_absolute_error: 0.4750
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 164ms/step - loss: 0.5966 - mean_absolute_error: 0.5035 - val_loss: 0.2174 - val_mean_absolute_error: 0.3683
Epoch 52/400
1/1 [==============================] - 0s 160ms/step - loss: 0.3629 - mean_absolute_error: 0.4245 - val_loss: 0.1992 - val_mean_absolute_error: 0.3583
Epoch 53/400
1/1 [==============================] - 0s 162ms/step - loss: 0.3785 - mean_absolute_error: 0.4283 - val_loss: 0.2652 - val_mean_absolute_error: 0.4123
Epoch 54/400
1/1 [==============================] - 0s 160ms/step - loss: 0.3233 - mean_absolute_error: 0.4033 - val_loss: 0.2064 - val_mean_absolute_error: 0.3551
Epoch 55/400
1/1 [==============================] - 0s 159ms/step - loss: 0.4979 - mean_absolute_error: 0.4542 - val_loss: 0.2394 - val_mean_absolute_error: 0.3872
Epoch 56/400
1/1 [==============================] - 0s 160ms/step - loss: 0.3889 - mean_absolute_error: 0.4481 - val_loss: 0.2280 - val_mean_absolute_error: 0.3600
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 168ms/step - loss: 0.5895 - mean_absolute_error: 0.4603 - val_loss: 0.2239 - val_mean_absolute_error: 0.3470
Epoch 102/400
1/1 [==============================] - 0s 170ms/step - loss: 0.2807 - mean_absolute_error: 0.3951 - val_loss: 0.2183 - val_mean_absolute_error: 0.3546
Epoch 103/400
1/1 [==============================] - 0s 159ms/step - loss: 0.4829 - mean_absolute_error: 0.4450 - val_loss: 0.1980 - val_mean_absolute_error: 0.3479
Epoch 104/400
1/1 [==============================] - 0s 161ms/step - loss: 0.2939 - mean_absolute_error: 0.3937 - val_loss: 0.2441 - val_mean_absolute_error: 0.3858
Epoch 105/400
1/1 [==============================] - 0s 158ms/step - loss: 0.2374 - mean_absolute_error: 0.3683 - val_loss: 0.2613 - val_mean_absolute_error: 0.3736
Epoch 106/400
1/1 [==============================] - 0s 161ms/step - loss: 0.4442 - mean_absolute_error: 0.4484 - val_loss: 0.2208 - val_mean_absolute_error: 0.3536
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 161ms/step - loss: 0.3309 - mean_absolute_error: 0.3618 - val_loss: 0.2136 - val_mean_absolute_error: 0.3505
Epoch 152/400
1/1 [==============================] - 0s 164ms/step - loss: 0.3692 - mean_absolute_error: 0.3877 - val_loss: 0.2141 - val_mean_absolute_error: 0.3555
Epoch 153/400
1/1 [==============================] - 0s 170ms/step - loss: 0.2864 - mean_absolute_error: 0.3809 - val_loss: 0.2276 - val_mean_absolute_error: 0.3383
Epoch 154/400
1/1 [==============================] - 0s 166ms/step - loss: 0.3295 - mean_absolute_error: 0.3453 - val_loss: 0.2727 - val_mean_absolute_error: 0.3878
Epoch 155/400
1/1 [==============================] - 0s 163ms/step - loss: 0.4512 - mean_absolute_error: 0.4068 - val_loss: 0.2571 - val_mean_absolute_error: 0.3860
Epoch 156/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1805 - mean_absolute_error: 0.3070 - val_loss: 0.2610 - val_mean_absolute_error: 0.3775
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 160ms/step - loss: 0.2506 - mean_absolute_error: 0.3581 - val_loss: 0.1977 - val_mean_absolute_error: 0.3420
Epoch 202/400
1/1 [==============================] - 0s 160ms/step - loss: 0.2552 - mean_absolute_error: 0.3683 - val_loss: 0.2080 - val_mean_absolute_error: 0.3429
Epoch 203/400
1/1 [==============================] - 0s 162ms/step - loss: 0.2030 - mean_absolute_error: 0.3204 - val_loss: 0.1944 - val_mean_absolute_error: 0.3479
Epoch 204/400
1/1 [==============================] - 0s 165ms/step - loss: 0.2577 - mean_absolute_error: 0.3267 - val_loss: 0.2531 - val_mean_absolute_error: 0.3879
Epoch 205/400
1/1 [==============================] - 0s 174ms/step - loss: 0.2630 - mean_absolute_error: 0.3656 - val_loss: 0.2167 - val_mean_absolute_error: 0.3581
Epoch 206/400
1/1 [==============================] - 0s 170ms/step - loss: 0.2592 - mean_absolute_error: 0.3575 - val_loss: 0.2326 - val_mean_absolute_error: 0.3693
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 159ms/step - loss: 0.1900 - mean_absolute_error: 0.3261 - val_loss: 0.2393 - val_mean_absolute_error: 0.3852
Epoch 252/400
1/1 [==============================] - 0s 159ms/step - loss: 0.3352 - mean_absolute_error: 0.3571 - val_loss: 0.2033 - val_mean_absolute_error: 0.3501
Epoch 253/400
1/1 [==============================] - 0s 159ms/step - loss: 0.1926 - mean_absolute_error: 0.2920 - val_loss: 0.1491 - val_mean_absolute_error: 0.2983
Epoch 254/400
1/1 [==============================] - 0s 160ms/step - loss: 0.2243 - mean_absolute_error: 0.3070 - val_loss: 0.1862 - val_mean_absolute_error: 0.3257
Epoch 255/400
1/1 [==============================] - 0s 163ms/step - loss: 0.2343 - mean_absolute_error: 0.3272 - val_loss: 0.1889 - val_mean_absolute_error: 0.3453
Epoch 256/400
1/1 [==============================] - 0s 170ms/step - loss: 0.3044 - mean_absolute_error: 0.3587 - val_loss: 0.1867 - val_mean_absolute_error: 0.3364
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 162ms/step - loss: 0.2268 - mean_absolute_error: 0.3312 - val_loss: 0.1816 - val_mean_absolute_error: 0.3391
Epoch 302/400
1/1 [==============================] - 0s 159ms/step - loss: 0.2408 - mean_absolute_error: 0.3178 - val_loss: 0.1894 - val_mean_absolute_error: 0.3259
Epoch 303/400
1/1 [==============================] - 0s 161ms/step - loss: 0.3148 - mean_absolute_error: 0.3313 - val_loss: 0.1925 - val_mean_absolute_error: 0.3370
Epoch 304/400
1/1 [==============================] - 0s 163ms/step - loss: 0.3316 - mean_absolute_error: 0.3647 - val_loss: 0.1873 - val_mean_absolute_error: 0.3520
Epoch 305/400
1/1 [==============================] - 0s 160ms/step - loss: 0.1895 - mean_absolute_error: 0.3167 - val_loss: 0.2033 - val_mean_absolute_error: 0.3531
Epoch 306/400
1/1 [==============================] - 0s 160ms/step - loss: 0.2989 - mean_absolute_error: 0.3166 - val_loss: 0.1782 - val_mean_absolute_error: 0.3283
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 162ms/step - loss: 0.1451 - mean_absolute_error: 0.2822 - val_loss: 0.1880 - val_mean_absolute_error: 0.3388
Epoch 352/400
1/1 [==============================] - 0s 165ms/step - loss: 0.1591 - mean_absolute_error: 0.2788 - val_loss: 0.2183 - val_mean_absolute_error: 0.3615
Epoch 353/400
1/1 [==============================] - 0s 165ms/step - loss: 0.2170 - mean_absolute_error: 0.3268 - val_loss: 0.2190 - val_mean_absolute_error: 0.3655
Epoch 354/400
1/1 [==============================] - 0s 171ms/step - loss: 0.2148 - mean_absolute_error: 0.3065 - val_loss: 0.2341 - val_mean_absolute_error: 0.3739
Epoch 355/400
1/1 [==============================] - 0s 160ms/step - loss: 0.2861 - mean_absolute_error: 0.3349 - val_loss: 0.1894 - val_mean_absolute_error: 0.3511
Epoch 356/400
1/1 [==============================] - 0s 160ms/step - loss: 0.3251 - mean_absolute_error: 0.3475 - val_loss: 0.1924 - val_mean_absolute_error: 0.3549
Epoch 357/

100/100 [==============================] - 6s 64ms/step - loss: 0.1853 - mean_absolute_error: 0.3351
val performance = [0.1853366494178772, 0.33505865931510925]
test performance =  [0.2168051153421402, 0.3187216818332672]


In [74]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [75]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
_performance['Multi'] = multi_window.compa(multi_linear_model, plot_col=out_features[0])

column : 클로로필-a
50.0


In [77]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
multi_window.plot24(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
multi_window.plot24_class(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Elman_RNN**

In [80]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [81]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [82]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 0.6613 - mean_absolute_error: 0.5688 - val_loss: 0.2688 - val_mean_absolute_error: 0.4494
Epoch 2/400
1/1 [==============================] - 0s 368ms/step - loss: 0.9733 - mean_absolute_error: 0.6485 - val_loss: 0.3002 - val_mean_absolute_error: 0.4701
Epoch 3/400
1/1 [==============================] - 0s 373ms/step - loss: 0.9409 - mean_absolute_error: 0.5791 - val_loss: 0.2750 - val_mean_absolute_error: 0.4272
Epoch 4/400
1/1 [==============================] - 0s 375ms/step - loss: 0.4136 - mean_absolute_error: 0.4335 - val_loss: 0.2280 - val_mean_absolute_error: 0.3952
Epoch 5/400
1/1 [==============================] - 0s 368ms/step - loss: 0.4296 - mean_absolute_error: 0.4216 - val_loss: 0.1764 - val_mean_absolute_error: 0.3169
Epoch 6/400
1/1 [==============================] - 0s 380ms/step - loss: 0.2513 - mean_absolute_error: 0.3695 - val_loss: 0.2186 - val_mean_absolute_error: 0.3463
Epoch 7/400
1/1 [========

1/1 [==============================] - 0s 366ms/step - loss: 0.2313 - mean_absolute_error: 0.2794 - val_loss: 0.1204 - val_mean_absolute_error: 0.2585
Epoch 52/400
1/1 [==============================] - 0s 383ms/step - loss: 0.1709 - mean_absolute_error: 0.2888 - val_loss: 0.1172 - val_mean_absolute_error: 0.2569
Epoch 53/400
1/1 [==============================] - 0s 372ms/step - loss: 0.0927 - mean_absolute_error: 0.2271 - val_loss: 0.1017 - val_mean_absolute_error: 0.2356
Epoch 54/400
1/1 [==============================] - 0s 376ms/step - loss: 0.1545 - mean_absolute_error: 0.2408 - val_loss: 0.1030 - val_mean_absolute_error: 0.2272
Epoch 55/400
1/1 [==============================] - 0s 370ms/step - loss: 0.1612 - mean_absolute_error: 0.2297 - val_loss: 0.1322 - val_mean_absolute_error: 0.2825
Epoch 56/400
1/1 [==============================] - 0s 370ms/step - loss: 0.1473 - mean_absolute_error: 0.2583 - val_loss: 0.1452 - val_mean_absolute_error: 0.2870
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 368ms/step - loss: 0.2396 - mean_absolute_error: 0.2784 - val_loss: 0.1331 - val_mean_absolute_error: 0.2638
Epoch 102/400
1/1 [==============================] - 0s 361ms/step - loss: 0.1544 - mean_absolute_error: 0.2578 - val_loss: 0.1706 - val_mean_absolute_error: 0.2908
Epoch 103/400
1/1 [==============================] - 0s 387ms/step - loss: 0.1317 - mean_absolute_error: 0.2290 - val_loss: 0.1234 - val_mean_absolute_error: 0.2507
Epoch 104/400
1/1 [==============================] - 0s 367ms/step - loss: 0.2350 - mean_absolute_error: 0.2693 - val_loss: 0.1005 - val_mean_absolute_error: 0.2284
Epoch 105/400
1/1 [==============================] - 0s 360ms/step - loss: 0.0974 - mean_absolute_error: 0.2202 - val_loss: 0.0993 - val_mean_absolute_error: 0.2353
Epoch 106/400
1/1 [==============================] - 0s 373ms/step - loss: 0.0740 - mean_absolute_error: 0.1993 - val_loss: 0.0948 - val_mean_absolute_error: 0.2342
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 380ms/step - loss: 0.0971 - mean_absolute_error: 0.2131 - val_loss: 0.1054 - val_mean_absolute_error: 0.2436
Epoch 152/400
1/1 [==============================] - 0s 360ms/step - loss: 0.0731 - mean_absolute_error: 0.1896 - val_loss: 0.0897 - val_mean_absolute_error: 0.2197
Epoch 153/400
1/1 [==============================] - 0s 371ms/step - loss: 0.0974 - mean_absolute_error: 0.2073 - val_loss: 0.0922 - val_mean_absolute_error: 0.2220
Epoch 154/400
1/1 [==============================] - 0s 375ms/step - loss: 0.1078 - mean_absolute_error: 0.2213 - val_loss: 0.1260 - val_mean_absolute_error: 0.2632
Epoch 155/400
1/1 [==============================] - 0s 382ms/step - loss: 0.2032 - mean_absolute_error: 0.2530 - val_loss: 0.1008 - val_mean_absolute_error: 0.2385
Epoch 156/400
1/1 [==============================] - 0s 375ms/step - loss: 0.1046 - mean_absolute_error: 0.2150 - val_loss: 0.1269 - val_mean_absolute_error: 0.2630
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 381ms/step - loss: 0.1413 - mean_absolute_error: 0.2429 - val_loss: 0.0987 - val_mean_absolute_error: 0.2308
Epoch 202/400
1/1 [==============================] - 0s 383ms/step - loss: 0.1808 - mean_absolute_error: 0.2457 - val_loss: 0.1108 - val_mean_absolute_error: 0.2473
Epoch 203/400
1/1 [==============================] - 0s 366ms/step - loss: 0.1065 - mean_absolute_error: 0.2170 - val_loss: 0.1026 - val_mean_absolute_error: 0.2322
Epoch 204/400
1/1 [==============================] - 0s 374ms/step - loss: 0.1060 - mean_absolute_error: 0.2025 - val_loss: 0.0996 - val_mean_absolute_error: 0.2317
Epoch 205/400
1/1 [==============================] - 0s 364ms/step - loss: 0.0915 - mean_absolute_error: 0.2006 - val_loss: 0.1192 - val_mean_absolute_error: 0.2500
Epoch 206/400
1/1 [==============================] - 0s 371ms/step - loss: 0.0823 - mean_absolute_error: 0.2018 - val_loss: 0.0962 - val_mean_absolute_error: 0.2273
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 373ms/step - loss: 0.1172 - mean_absolute_error: 0.2169 - val_loss: 0.0999 - val_mean_absolute_error: 0.2136
Epoch 252/400
1/1 [==============================] - 0s 361ms/step - loss: 0.1299 - mean_absolute_error: 0.2240 - val_loss: 0.1088 - val_mean_absolute_error: 0.2325
Epoch 253/400
1/1 [==============================] - 0s 372ms/step - loss: 0.1104 - mean_absolute_error: 0.1988 - val_loss: 0.0915 - val_mean_absolute_error: 0.2196
Epoch 254/400
1/1 [==============================] - 0s 373ms/step - loss: 0.1370 - mean_absolute_error: 0.2124 - val_loss: 0.1001 - val_mean_absolute_error: 0.2227
Epoch 255/400
1/1 [==============================] - 0s 361ms/step - loss: 0.0836 - mean_absolute_error: 0.1960 - val_loss: 0.0875 - val_mean_absolute_error: 0.2131
Epoch 256/400
1/1 [==============================] - 0s 368ms/step - loss: 0.0984 - mean_absolute_error: 0.1918 - val_loss: 0.1277 - val_mean_absolute_error: 0.2566
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 370ms/step - loss: 0.0696 - mean_absolute_error: 0.1751 - val_loss: 0.1506 - val_mean_absolute_error: 0.2803
Epoch 302/400
1/1 [==============================] - 0s 363ms/step - loss: 0.1474 - mean_absolute_error: 0.2233 - val_loss: 0.0951 - val_mean_absolute_error: 0.2248
Epoch 303/400
1/1 [==============================] - 0s 377ms/step - loss: 0.0843 - mean_absolute_error: 0.1909 - val_loss: 0.1262 - val_mean_absolute_error: 0.2448
Epoch 304/400
1/1 [==============================] - 0s 381ms/step - loss: 0.1215 - mean_absolute_error: 0.2099 - val_loss: 0.1148 - val_mean_absolute_error: 0.2464
Epoch 305/400
1/1 [==============================] - 0s 377ms/step - loss: 0.0908 - mean_absolute_error: 0.1877 - val_loss: 0.0987 - val_mean_absolute_error: 0.2206
Epoch 306/400
1/1 [==============================] - 0s 355ms/step - loss: 0.1361 - mean_absolute_error: 0.2206 - val_loss: 0.1303 - val_mean_absolute_error: 0.2536
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 379ms/step - loss: 0.0879 - mean_absolute_error: 0.1993 - val_loss: 0.1031 - val_mean_absolute_error: 0.2425
Epoch 352/400
1/1 [==============================] - 0s 378ms/step - loss: 0.0665 - mean_absolute_error: 0.1778 - val_loss: 0.0957 - val_mean_absolute_error: 0.2340
Epoch 353/400
1/1 [==============================] - 0s 371ms/step - loss: 0.0720 - mean_absolute_error: 0.1887 - val_loss: 0.1162 - val_mean_absolute_error: 0.2443
Epoch 354/400
1/1 [==============================] - 0s 363ms/step - loss: 0.1818 - mean_absolute_error: 0.2247 - val_loss: 0.1178 - val_mean_absolute_error: 0.2490
Epoch 355/400
1/1 [==============================] - 0s 361ms/step - loss: 0.1011 - mean_absolute_error: 0.2016 - val_loss: 0.1022 - val_mean_absolute_error: 0.2396
Epoch 356/400
1/1 [==============================] - 0s 370ms/step - loss: 0.1149 - mean_absolute_error: 0.2226 - val_loss: 0.0956 - val_mean_absolute_error: 0.2303
Epoch 357/

100/100 [==============================] - 12s 122ms/step - loss: 0.1030 - mean_absolute_error: 0.2066
val performance = [0.10962076485157013, 0.24079762399196625]
test performance =  [0.10295674949884415, 0.20655457675457]


In [83]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
_performance['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0])

column : 클로로필-a
80.46875


In [85]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
multi_window.plot24(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
multi_window.plot24_class(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [88]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [89]:
MAX_EPOCHS = 100

In [90]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/100
1/1 [==============================] - 1s 911ms/step - loss: 0.9373 - mean_absolute_error: 0.6358 - val_loss: 0.3705 - val_mean_absolute_error: 0.5225
Epoch 2/100
1/1 [==============================] - 0s 262ms/step - loss: 0.4946 - mean_absolute_error: 0.5156 - val_loss: 0.3196 - val_mean_absolute_error: 0.4925
Epoch 3/100
1/1 [==============================] - 0s 271ms/step - loss: 0.6196 - mean_absolute_error: 0.5662 - val_loss: 0.3254 - val_mean_absolute_error: 0.4956
Epoch 4/100
1/1 [==============================] - 0s 256ms/step - loss: 0.3734 - mean_absolute_error: 0.4690 - val_loss: 0.2996 - val_mean_absolute_error: 0.4710
Epoch 5/100
1/1 [==============================] - 0s 255ms/step - loss: 0.7094 - mean_absolute_error: 0.5897 - val_loss: 0.3109 - val_mean_absolute_error: 0.4729
Epoch 6/100
1/1 [==============================] - 0s 259ms/step - loss: 0.4747 - mean_absolute_error: 0.4922 - val_loss: 0.2641 - val_mean_absolute_error: 0.4335
Epoch 7/100
1/1 [=====

1/1 [==============================] - 0s 258ms/step - loss: 0.2547 - mean_absolute_error: 0.2992 - val_loss: 0.1084 - val_mean_absolute_error: 0.2481
Epoch 52/100
1/1 [==============================] - 0s 258ms/step - loss: 0.3033 - mean_absolute_error: 0.3244 - val_loss: 0.0944 - val_mean_absolute_error: 0.2345
Epoch 53/100
1/1 [==============================] - 0s 256ms/step - loss: 0.1633 - mean_absolute_error: 0.2755 - val_loss: 0.1002 - val_mean_absolute_error: 0.2350
Epoch 54/100
1/1 [==============================] - 0s 266ms/step - loss: 0.1142 - mean_absolute_error: 0.2359 - val_loss: 0.1075 - val_mean_absolute_error: 0.2515
Epoch 55/100
1/1 [==============================] - 0s 260ms/step - loss: 0.1960 - mean_absolute_error: 0.2795 - val_loss: 0.1009 - val_mean_absolute_error: 0.2458
Epoch 56/100
1/1 [==============================] - 0s 258ms/step - loss: 0.1924 - mean_absolute_error: 0.2814 - val_loss: 0.0994 - val_mean_absolute_error: 0.2428
Epoch 57/100
1/1 [===========

In [91]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
_performance['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0])

column : 클로로필-a
71.09375


In [93]:
multi_window.plot2(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
multi_window.plot24_class(gru_model, plot_col=out_features[0])

/home/jaewon/venv379/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**LSTM**

In [96]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [97]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.4217 - mean_absolute_error: 0.5018 - val_loss: 0.3107 - val_mean_absolute_error: 0.4815
Epoch 2/100
1/1 [==============================] - 0s 458ms/step - loss: 0.6230 - mean_absolute_error: 0.5416 - val_loss: 0.3653 - val_mean_absolute_error: 0.5313
Epoch 3/100
1/1 [==============================] - 0s 476ms/step - loss: 0.3819 - mean_absolute_error: 0.4778 - val_loss: 0.3390 - val_mean_absolute_error: 0.4909
Epoch 4/100
1/1 [==============================] - 0s 473ms/step - loss: 0.6025 - mean_absolute_error: 0.5839 - val_loss: 0.3501 - val_mean_absolute_error: 0.5013
Epoch 5/100
1/1 [==============================] - 0s 500ms/step - loss: 1.5471 - mean_absolute_error: 0.7737 - val_loss: 0.2897 - val_mean_absolute_error: 0.4579
Epoch 6/100
1/1 [==============================] - 0s 468ms/step - loss: 0.5988 - mean_absolute_error: 0.5311 - val_loss: 0.2548 - val_mean_absolute_error: 0.4221
Epoch 7/100
1/1 [========

1/1 [==============================] - 0s 457ms/step - loss: 0.1516 - mean_absolute_error: 0.2538 - val_loss: 0.1061 - val_mean_absolute_error: 0.2439
Epoch 52/100
1/1 [==============================] - 0s 459ms/step - loss: 0.1031 - mean_absolute_error: 0.2185 - val_loss: 0.0938 - val_mean_absolute_error: 0.2306
Epoch 53/100
1/1 [==============================] - 0s 472ms/step - loss: 0.1542 - mean_absolute_error: 0.2621 - val_loss: 0.1076 - val_mean_absolute_error: 0.2555
Epoch 54/100
1/1 [==============================] - 0s 484ms/step - loss: 0.1325 - mean_absolute_error: 0.2466 - val_loss: 0.0999 - val_mean_absolute_error: 0.2411
Epoch 55/100
1/1 [==============================] - 0s 460ms/step - loss: 0.2220 - mean_absolute_error: 0.2798 - val_loss: 0.0934 - val_mean_absolute_error: 0.2306
Epoch 56/100
1/1 [==============================] - 0s 463ms/step - loss: 0.1558 - mean_absolute_error: 0.2464 - val_loss: 0.0994 - val_mean_absolute_error: 0.2395
Epoch 57/100
1/1 [===========

In [98]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
_performance['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0])

column : 클로로필-a
76.5625


In [100]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
multi_window.plot24(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
multi_window.plot24_class(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**CNN model**

In [103]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [104]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/400
1/1 [==============================] - 0s 421ms/step - loss: 1.0252 - mean_absolute_error: 0.6301 - val_loss: 0.3447 - val_mean_absolute_error: 0.5159
Epoch 2/400
1/1 [==============================] - 0s 158ms/step - loss: 0.5940 - mean_absolute_error: 0.5528 - val_loss: 0.2801 - val_mean_absolute_error: 0.4484
Epoch 3/400
1/1 [==============================] - 0s 156ms/step - loss: 0.4204 - mean_absolute_error: 0.4661 - val_loss: 0.2228 - val_mean_absolute_error: 0.4136
Epoch 4/400
1/1 [==============================] - 0s 159ms/step - loss: 0.4764 - mean_absolute_error: 0.4900 - val_loss: 0.2106 - val_mean_absolute_error: 0.3555
Epoch 5/400
1/1 [==============================] - 0s 156ms/step - loss: 0.3517 - mean_absolute_error: 0.4256 - val_loss: 0.2195 - val_mean_absolute_error: 0.3536
Epoch 6/400
1/1 [==============================] - 0s 161ms/step - loss: 0.2461 - mean_absolute_error: 0.3401 - val_loss: 0.2399 - val_mean_absolute_error: 0.3707
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 165ms/step - loss: 0.2332 - mean_absolute_error: 0.2730 - val_loss: 0.0729 - val_mean_absolute_error: 0.2008
Epoch 52/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1258 - mean_absolute_error: 0.2256 - val_loss: 0.0806 - val_mean_absolute_error: 0.2049
Epoch 53/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1325 - mean_absolute_error: 0.2318 - val_loss: 0.0786 - val_mean_absolute_error: 0.2052
Epoch 54/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1080 - mean_absolute_error: 0.2166 - val_loss: 0.0833 - val_mean_absolute_error: 0.2072
Epoch 55/400
1/1 [==============================] - 0s 159ms/step - loss: 0.1691 - mean_absolute_error: 0.2281 - val_loss: 0.0881 - val_mean_absolute_error: 0.2150
Epoch 56/400
1/1 [==============================] - 0s 157ms/step - loss: 0.2711 - mean_absolute_error: 0.2729 - val_loss: 0.0861 - val_mean_absolute_error: 0.2135
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 155ms/step - loss: 0.1185 - mean_absolute_error: 0.2140 - val_loss: 0.0660 - val_mean_absolute_error: 0.1834
Epoch 102/400
1/1 [==============================] - 0s 170ms/step - loss: 0.1992 - mean_absolute_error: 0.2447 - val_loss: 0.0669 - val_mean_absolute_error: 0.1937
Epoch 103/400
1/1 [==============================] - 0s 167ms/step - loss: 0.1809 - mean_absolute_error: 0.2321 - val_loss: 0.0588 - val_mean_absolute_error: 0.1750
Epoch 104/400
1/1 [==============================] - 0s 168ms/step - loss: 0.1744 - mean_absolute_error: 0.2388 - val_loss: 0.0666 - val_mean_absolute_error: 0.1808
Epoch 105/400
1/1 [==============================] - 0s 160ms/step - loss: 0.0860 - mean_absolute_error: 0.1992 - val_loss: 0.0658 - val_mean_absolute_error: 0.1930
Epoch 106/400
1/1 [==============================] - 0s 158ms/step - loss: 0.2069 - mean_absolute_error: 0.2448 - val_loss: 0.0682 - val_mean_absolute_error: 0.1865
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 158ms/step - loss: 0.1889 - mean_absolute_error: 0.2594 - val_loss: 0.0772 - val_mean_absolute_error: 0.2130
Epoch 152/400
1/1 [==============================] - 0s 158ms/step - loss: 0.2707 - mean_absolute_error: 0.2620 - val_loss: 0.0742 - val_mean_absolute_error: 0.1972
Epoch 153/400
1/1 [==============================] - 0s 155ms/step - loss: 0.1312 - mean_absolute_error: 0.2361 - val_loss: 0.0626 - val_mean_absolute_error: 0.1890
Epoch 154/400
1/1 [==============================] - 0s 157ms/step - loss: 0.0788 - mean_absolute_error: 0.1897 - val_loss: 0.0769 - val_mean_absolute_error: 0.1996
Epoch 155/400
1/1 [==============================] - 0s 159ms/step - loss: 0.1734 - mean_absolute_error: 0.2427 - val_loss: 0.0734 - val_mean_absolute_error: 0.1896
Epoch 156/400
1/1 [==============================] - 0s 178ms/step - loss: 0.1450 - mean_absolute_error: 0.2147 - val_loss: 0.0860 - val_mean_absolute_error: 0.2134
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 157ms/step - loss: 0.0861 - mean_absolute_error: 0.2012 - val_loss: 0.0582 - val_mean_absolute_error: 0.1742
Epoch 202/400
1/1 [==============================] - 0s 162ms/step - loss: 0.1376 - mean_absolute_error: 0.2293 - val_loss: 0.0717 - val_mean_absolute_error: 0.1874
Epoch 203/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1096 - mean_absolute_error: 0.2090 - val_loss: 0.0650 - val_mean_absolute_error: 0.1882
Epoch 204/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1067 - mean_absolute_error: 0.2106 - val_loss: 0.0707 - val_mean_absolute_error: 0.1876
Epoch 205/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1014 - mean_absolute_error: 0.2129 - val_loss: 0.0810 - val_mean_absolute_error: 0.2123
Epoch 206/400
1/1 [==============================] - 0s 155ms/step - loss: 0.1776 - mean_absolute_error: 0.2499 - val_loss: 0.0730 - val_mean_absolute_error: 0.2057
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 157ms/step - loss: 0.0919 - mean_absolute_error: 0.2002 - val_loss: 0.0587 - val_mean_absolute_error: 0.1776
Epoch 252/400
1/1 [==============================] - 0s 159ms/step - loss: 0.0924 - mean_absolute_error: 0.1944 - val_loss: 0.0589 - val_mean_absolute_error: 0.1758
Epoch 253/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1021 - mean_absolute_error: 0.1994 - val_loss: 0.0749 - val_mean_absolute_error: 0.2004
Epoch 254/400
1/1 [==============================] - 0s 159ms/step - loss: 0.1737 - mean_absolute_error: 0.2378 - val_loss: 0.0759 - val_mean_absolute_error: 0.1989
Epoch 255/400
1/1 [==============================] - 0s 154ms/step - loss: 0.0777 - mean_absolute_error: 0.1931 - val_loss: 0.0781 - val_mean_absolute_error: 0.2049
Epoch 256/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2029 - mean_absolute_error: 0.2475 - val_loss: 0.0547 - val_mean_absolute_error: 0.1680
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 163ms/step - loss: 0.0747 - mean_absolute_error: 0.1809 - val_loss: 0.0653 - val_mean_absolute_error: 0.1867
Epoch 302/400
1/1 [==============================] - 0s 157ms/step - loss: 0.0774 - mean_absolute_error: 0.1884 - val_loss: 0.0774 - val_mean_absolute_error: 0.2111
Epoch 303/400
1/1 [==============================] - 0s 157ms/step - loss: 0.0821 - mean_absolute_error: 0.1937 - val_loss: 0.0840 - val_mean_absolute_error: 0.2114
Epoch 304/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1044 - mean_absolute_error: 0.2040 - val_loss: 0.0816 - val_mean_absolute_error: 0.2064
Epoch 305/400
1/1 [==============================] - 0s 166ms/step - loss: 0.1155 - mean_absolute_error: 0.2107 - val_loss: 0.0729 - val_mean_absolute_error: 0.1979
Epoch 306/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1110 - mean_absolute_error: 0.2159 - val_loss: 0.0641 - val_mean_absolute_error: 0.1851
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 175ms/step - loss: 0.0718 - mean_absolute_error: 0.1831 - val_loss: 0.0759 - val_mean_absolute_error: 0.2067
Epoch 352/400
1/1 [==============================] - 0s 164ms/step - loss: 0.0851 - mean_absolute_error: 0.1973 - val_loss: 0.0647 - val_mean_absolute_error: 0.1928
Epoch 353/400
1/1 [==============================] - 0s 171ms/step - loss: 0.0972 - mean_absolute_error: 0.2019 - val_loss: 0.0685 - val_mean_absolute_error: 0.1993
Epoch 354/400
1/1 [==============================] - 0s 167ms/step - loss: 0.1863 - mean_absolute_error: 0.2399 - val_loss: 0.0739 - val_mean_absolute_error: 0.2052
Epoch 355/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1274 - mean_absolute_error: 0.2327 - val_loss: 0.0666 - val_mean_absolute_error: 0.1921
Epoch 356/400
1/1 [==============================] - 0s 160ms/step - loss: 0.0727 - mean_absolute_error: 0.1829 - val_loss: 0.0724 - val_mean_absolute_error: 0.1980
Epoch 357/

1/1 [==============================] - 0s 81ms/step - loss: 0.0669 - mean_absolute_error: 0.1843
val performance = [0.06687019020318985, 0.18425005674362183]
test performance =  [0.14660614728927612, 0.23732338845729828]


In [105]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 79ms/step - loss: 0.0763 - mean_absolute_error: 0.2022
val performance = [0.07630806416273117, 0.20220869779586792]
test performance =  [0.07934088259935379, 0.19372017681598663]


In [106]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
out_features = [4]
out_features[0]

4

In [108]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
multi_window.plot24(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
multi_window.plot24_class(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: '탁도'

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

In [ ]:
print(_performance)

In [ ]:
out_features = [1]

_performance['base'] = multi_window.compa(last_baseline, plot_col=out_features[0])
_performance['Multi'] = multi_window.compa(multi_linear_model, plot_col=out_features[0])
_performance['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0])
_performance['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0])
_performance['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0])


print(_performance)

In [ ]:
x = np.arange(len(_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in _performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

**Autoregressive model**

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)
    
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)
  #print('x =', x)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  predictions = tf.keras.layers.Lambda(lambda x: x[:, :, out_features[0]:(out_features[0]+len(out_features))])(predictions)
  return predictions

FeedBack.call = call

print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

print('val performance =', multi_val_performance['AR LSTM'])
print('test performance = ', multi_performance['AR LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot24(feedback_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()